# Residence Clustering

Start with missions from 1850 to 1940

In [1]:
import pandas as pd

In [7]:
df = pd.read_csv("missions_1850_to_1940.csv", encoding="ISO-8859-1")
# df = pd.read_csv("missions_1850_to_1940_utf8.csv", encoding="utf-8")

df["year"] = df["year"].apply(lambda x:  int(x)) # just in case 
df.head()

,name,birth_place,mission,loc_served,year,type,residence,url
0,Lyman Palmer Pinkston,"Salt Lake City, Salt Lake, Utah",Central States Mission,Central States,1931,Proselytizing,"Glendale, California",https://history.churchofjesuschrist.org/chd/in...
1,Nelson Daniel Russ,"Alabama, Genesee, New York",Eastern States Mission,Eastern States,1898,Proselytizing,"Wilford, Fremont, Idaho, United States",https://history.churchofjesuschrist.org/chd/in...
2,Preston Baxter Maughan,"Wellsville, Cache, Utah",Northwestern States (Pacific) Mission,Northwestern States,1919,NaN,"Wellsville, Cache, Utah, United States",https://history.churchofjesuschrist.org/chd/in...
3,Joseph Ezra Wood,"Holden, Millard, Utah",British Mission,British,1912,NaN,"Holden, Millard, Utah, United States",https://history.churchofjesuschrist.org/chd/in...
4,Jens Wilhelm Olsen,"Asnas, Oshered, Denmark",Danish Mission,Danish,1935,NaN,United States,https://history.churchofjesuschrist.org/chd/in...


In [3]:
def count_unique(input_list):
    mission_counts = {}

    for i in range(len(input_list)):
        cur_mission = input_list[i]
        if cur_mission in mission_counts:
            mission_counts[cur_mission] += 1
        else:
            mission_counts[cur_mission] = 1
            
    return mission_counts

def dict_sort(count_dict):
    mission_sort = []
    for k, v in count_dict.items():
        mission_sort.append((v, k)) # count, then mission name
    mission_sort.sort(reverse=True)

    return mission_sort 


**TODO**:  
* For now, ignore nan, just Utah, United Staes, or anything outside Utah
* If residence is a "mission", then check to see if it was a mission transfer & use prior departure residence 

In [8]:
# mission clean-up
# if residence contians "mission", check previous row
# if same missionary name and previous residence does not contain "mission", replace it
# leave the rest as is 

df["new_residence"] = df['residence'].copy(deep=True)

for i in range(len(df)):
    cur_name = str(df["name"].iloc[i])
    cur_residence = str(df["new_residence"].iloc[i]).lower()
    if "mission" in cur_residence or "eastern states" in cur_residence or "german-austrian" in cur_residence:
        # check previous entry
        if i>0 and str(df["name"].iloc[i-1]) == cur_name:
            if not "mission" in str(df["new_residence"].iloc[i-1]).lower():
                df["new_residence"].iloc[i] = df["new_residence"].iloc[i-1]



C:\Users\10502190\AppData\Local\Temp\ipykernel_20048\2546446112.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["new_residence"].iloc[i] = df["new_residence"].iloc[i-1]


In [9]:
# Group by residence

import os 

# get current files

all_files = os.listdir("cities")

for file in all_files:
    with open(f"cities/{file}", "r", encoding='utf-8') as f:
        lines = f.readlines()
        lines = [l.strip() for l in lines]        
        city = lines[0] 
        names = set(lines)
        df["new_residence"] = df["new_residence"].apply(lambda x: city if str(x) in names else str(x))

In [11]:
df[:20]

,name,birth_place,mission,loc_served,year,type,residence,url,new_residence
0,Lyman Palmer Pinkston,"Salt Lake City, Salt Lake, Utah",Central States Mission,Central States,1931,Proselytizing,"Glendale, California",https://history.churchofjesuschrist.org/chd/in...,"Glendale, Los Angeles, California, United States"
1,Nelson Daniel Russ,"Alabama, Genesee, New York",Eastern States Mission,Eastern States,1898,Proselytizing,"Wilford, Fremont, Idaho, United States",https://history.churchofjesuschrist.org/chd/in...,"Wilford, Fremont, Idaho, United States"
2,Preston Baxter Maughan,"Wellsville, Cache, Utah",Northwestern States (Pacific) Mission,Northwestern States,1919,NaN,"Wellsville, Cache, Utah, United States",https://history.churchofjesuschrist.org/chd/in...,"Wellsville, Cache, Utah, United States"
3,Joseph Ezra Wood,"Holden, Millard, Utah",British Mission,British,1912,NaN,"Holden, Millard, Utah, United States",https://history.churchofjesuschrist.org/chd/in...,"Holden, Millard, Utah, United States"
4,Jens Wilhelm Olsen,"Asnas, Oshered, Denmark",Danish Mission,Danish,1935,NaN,United States,https://history.churchofjesuschrist.org/chd/in...,United States
5,Melvin Porter Callister,"Fillmore, Millard, Utah",Northwestern States (Pacific) Mission,Northwestern States,1919,NaN,"Salt Lake City, Salt Lake, Utah, United States",https://history.churchofjesuschrist.org/chd/in...,"Salt Lake City, Salt Lake, Utah, United States"
6,Emma Virginia Bodell,"Herriman, Salt Lake, Utah",Northwestern States (Pacific) Mission,Northwestern States,1913,Proselytizing,"Herriman, Salt Lake, Utah, United States",https://history.churchofjesuschrist.org/chd/in...,"Herriman, Salt Lake, Utah, United States"
7,Jedediah P Nelson,"Santaquin, Utah, Utah",Northern States Mission,Northern States,1900,NaN,"Santaquin, Utah, Utah, United States",https://history.churchofjesuschrist.org/chd/in...,"Santaquin, Utah, Utah, United States"
8,Andrew Selden Millward,"Grantsville, Tooele, Utah",Eastern States Mission,Eastern States,1924,NaN,"Berkely, California, United States",https://history.churchofjesuschrist.org/chd/in...,"Berkeley, Alameda, California, United States"
9,Kenneth Malcolm Sproul,"Washington, Washington, Utah",Eastern States Mission,Eastern States,1927,NaN,"Blackfoot, Bingham, Idaho, United States",https://history.churchofjesuschrist.org/chd/in...,"Blackfoot, Bingham, Idaho, United States"


In [63]:
# Check current residence groups

# get all residences
count_dict = count_unique(list(df["residence"]))

# get residences by mission
# mission = "Hawaiian Mission"
# count_dict = count_unique(list(df[df["mission"] == mission]["residence"]))

# get missions by residence
# residence = "Provo, Utah, Utah, United States"
# count_dict = count_unique(list(df[df["residence"] == residence]["mission"]))

sort_list = dict_sort(count_dict)

print(len(sort_list))

for item in sort_list: #[:10]:
    print(f"{item[1]}:  {item[0]}")  

1394
Salt Lake City, Salt Lake, Utah, United States:  6702
United States:  4187
Ogden, Weber, Utah, United States:  1470
nan:  1302
Logan, Cache, Utah, United States:  1032
Provo, Utah, Utah, United States:  1018
Brigham City, Box Elder, Utah, United States:  615
Preston, Oneida, Idaho, United States:  463
Bountiful, Davis, Utah, United States:  413
Lehi, Utah, Utah, United States:  396
Rexburg, Madison, Idaho, United States:  377
Murray, Salt Lake, Utah, United States:  370
Spanish Fork, Utah, Utah, United States:  334
Mesa, Maricopa, Arizona, United States:  308
Smithfield, Cache, Utah, United States:  304
American Fork, Utah, Utah, United States:  286
Richfield, Sevier, Utah, United States:  283
Pleasant Grove, Utah, Utah, United States:  280
Nephi, Juab, Utah, United States:  274
Hyrum, Cache, Utah, United States:  274
St George, Washington, Utah, United States:  266
Payson, Utah, Utah, United States:  262
Farmington, Davis, Utah, United States:  252
Vernal, Uintah, Utah, United St

In [17]:

results = []
for k in count_dict:
    cur_key = str(k).lower()
    
    #if "midvale".lower() in cur_key:
    #    results.append(k)
    
    # if cur_key[0] > "v":
    #     results.append(k)
    #if cur_key[0] == "s":
    #    if cur_key[1] > "u":
    #        results.append(k)
    
    results.append(k)
    
    
    
# export to file
"""
with open("cur_cities.txt", "w", encoding="utf-8") as f:
    for k in results:
        f.write(f"{k}\n")   
"""    
    
results.sort()
print(len(results))

for k in results:
    print(k)

1397
Aberdare, Glamorgan, Wales, United Kingdom
Aberdeen, Bingham, Idaho, United States
Abicel, Union, Oregon, United States
Abraham, Millard, Utah, United States
Acequia, Minidoka, Idaho, United States
Ada, Idaho, United States
Adamsville, Beaver, Utah, United States
Aetna, Cardston, Alberta, Canada
Afton, Lincoln, Wyoming, United States
Akron, Summit, Ohio, United States
Alameda, Bannock, Idaho, United States
Alameda, California, United States
Alamo, Lincoln, Nevada, United States
Alamosa, Alamosa, Colorado, United States
Alberta, Canada
Albion, Cassia, Idaho, United States
Albuquerque, Bernalillo, New Mexico, United States
Alhambra, Los Angeles, California, United States
Alicel, Union, Oregon, United States
Alliance, Jackson, Florida, United States
Allison, McKinley, New Mexico, United States
Alltwen, Glamorgan, Wales, United Kingdom
Alma, Maricopa, Arizona, United States
Almo, Cassia, Idaho, United States
Almy, Uinta, Wyoming, United States
Alpine, Apache, Arizona, United States
Al

In [11]:
# Export clean data

df.to_csv("mission_data_clean.csv", encoding="utf-8", index=False)
